In [1]:
import os
import pandas as pd
from datetime import datetime

def convert_csv_to_utdf_volume(input_dir, output_file):
    """
    Convert all CSV files in input_dir to a single UTDF volume format file.
    """
    # Define expected turn columns
    turn_columns = [
        "NBL", "NBT", "NBR",
        "SBL", "SBT", "SBR",
        "EBL", "EBT", "EBR",
        "WBL", "WBT", "WBR"
    ]

    all_data = []

    # Process each CSV file in directory
    for filename in os.listdir(input_dir):
        if filename.endswith(".csv") and "_Counts_" in filename:
            print(f"Processing: {filename}")

            # Extract INTID from filename (first part before first underscore)
            intid = filename.split("_")[0]

            # Load CSV
            filepath = os.path.join(input_dir, filename)
            df = pd.read_csv(filepath)

            # Ensure 'Time' column exists
            if 'Time' not in df.columns:
                print(f"Warning: 'Time' column missing in {filename}. Skipping.")
                continue

            # Convert Time to datetime (flexible parsing)
            df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
            if df['Time'].isnull().any():
                print(f"Warning: Some invalid datetime in {filename}. Rows with invalid time will be dropped.")
                df = df.dropna(subset=['Time'])

            # Create DATE (mm/dd/yyyy) and TIME (hhmm)
            df['DATE'] = df['Time'].dt.strftime('%m/%d/%Y')
            df['TIME'] = df['Time'].dt.strftime('%H%M').astype(int)  # UTDF often uses integer hhmm

            # Add INTID column
            df['INTID'] = intid

            # Filter to only needed columns
            required_cols = ['INTID', 'DATE', 'TIME'] + [col for col in turn_columns if col in df.columns]
            missing_turns = set(turn_columns) - set(df.columns)
            if missing_turns:
                print(f"Warning: Missing turn columns in {filename}: {missing_turns}")

            df_filtered = df[required_cols].copy()

            all_data.append(df_filtered)

    if not all_data:
        print("No valid data found.")
        return

    # Combine all data
    combined_df = pd.concat(all_data, ignore_index=True)

    # Write to UTDF format file
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        # Write UTDF header
        f.write("[UTDF VOLUME FILE]\n")
        f.write("VERSION=1.0\n")
        f.write("DESCRIPTION=Turning Movement Volumes\n")
        f.write(f"EXPORTED_ON={datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("\n")  # blank line before data

        # Write column headers
        f.write(",".join(combined_df.columns) + "\n")

        # Write data rows
        combined_df.to_csv(f, index=False, header=False, mode='a')

    print(f"✅ UTDF volume file written to: {output_file}")


In [2]:
INPUT_DIRECTORY = "C:\\Users\\rhansen\\OneDrive - Idaho Transportation Department\\Desktop\\Current Projects\\Franklin Caldwell\\Counts"  
OUTPUT_FILE = "output_volume.csv"  

convert_csv_to_utdf_volume(INPUT_DIRECTORY, OUTPUT_FILE)

Processing: 12_Counts_15min_2025_08_11-2025_09_06.csv
Processing: 15_Counts_15min_2025_08_10-2025_09_06.csv
Processing: 18_Counts_15min_2025_08_10-2025_09_06.csv
Processing: 21_Counts_15min_2025_08_26-2025_09_06.csv
Processing: 6_Counts_15min_2025_08_10-2025_09_06.csv
Processing: 9_Counts_15min_2025_08_10-2025_09_06.csv
✅ UTDF volume file written to: output_volume.csv
